In [1]:
from gcn import Model
from dataset import PopulationDataset
import numpy as np
import argparse
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from math import sqrt
from time import time


def train(model, train_loader, optimizer, criterion, epoch, label, label_idx, label_weight, theta):
    model.train()
    train_loss = 0
    for batch_idx, (data, x_t, target) in enumerate(train_loader):
        data, x_t, target = Variable(data).cuda(), Variable(x_t).cuda(), Variable(target).cuda()
        optimizer.zero_grad()
        output, y = model(data, x_t, label_idx)
        loss1 = criterion(output, target)
        loss2 = nn.functional.cross_entropy(y, label, weight=label_weight)
        loss = loss1 + loss2 * theta
        loss.backward()
        optimizer.step()
        if batch_idx % 20 == 0:
            print ("Epoch: %d [%d/%d (%.0f%%)]\tLoss: %.6f\tLabel Loss: %.6f" % (
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), sqrt(loss1.item()), loss2.item()))
        train_loss += loss1.item() * len(data)
    return sqrt(train_loss / len(train_loader.dataset))


def val(model, val_loader, criterion, epoch, label_idx):
    model.eval()
    val_loss = 0
    for batch_idx, (data, x_t, target) in enumerate(val_loader):
        data, x_t, target = Variable(data).cuda(), Variable(x_t).cuda(), Variable(target).cuda()
        output, _ = model(data, x_t, label_idx)
        loss = criterion(output, target)
        val_loss += loss.item() * len(data)
    print ("\nEpoch: %d \tVal Loss: %.6f" % (
        epoch, sqrt(val_loss / len(val_loader.dataset))))
    return sqrt(val_loss / len(val_loader.dataset))


def test(model, test_loader, criterion, epoch, label_idx):
    model.eval()
    test_loss = 0
    for batch_idx, (data, x_t, target) in enumerate(test_loader):
        data, x_t, target = Variable(data).cuda(), Variable(x_t).cuda(), Variable(target).cuda()
        output, _ = model(data, x_t, label_idx)
        loss = criterion(output, target)
        test_loss += loss.item() * len(data)
    print ("Epoch: %d \tTest Loss: %.6f\n" % (
        epoch, sqrt(test_loss / len(test_loader.dataset))))
    return sqrt(test_loss / len(test_loader.dataset))


parser = argparse.ArgumentParser(description='3D-DGCN')
parser.add_argument('--batch', type=int, default=64, metavar='B',
                    help='batch size for training')
parser.add_argument('--test-batch', type=int, default=32, metavar='TB',
                    help='batch size for testing')
parser.add_argument('--epoch', type=int, default=1600, metavar='E',
                    help='number of iterations')
parser.add_argument('--lr', type=float, default=0.001, metavar='LR',
                    help='learning rate')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed')
parser.add_argument('--division', type=str, default='irregular', metavar='D',
                    help='the spatial division (regular or irregular)')
parser.add_argument('--K', type=int, default='9', metavar='K',
                    help='number of partitions')
parser.add_argument('--C', type=int, default='2', metavar='C',
                    help='number of input channels')
parser.add_argument('--T', type=int, default='5', metavar='T',
                    help='number of time intervals in a training sample')
parser.add_argument('--theta', type=float, default='10', metavar='T',
                    help='hyper-parameter in loss function')
parser.add_argument('--load', type=int, default='0', metavar='L',
                    help='load checkpoint/epoch_x.tar (x>0)')
parser.add_argument('-f', type=str, default='')
args = parser.parse_args()

train_loader = torch.utils.data.DataLoader(PopulationDataset(division=args.division,
                                                             T=args.T,
                                                             type='train'),
                                           batch_size=args.test_batch,
                                           shuffle=True)

val_loader = torch.utils.data.DataLoader(PopulationDataset(division=args.division,
                                                           T=args.T,
                                                           type='val'),
                                         batch_size=args.test_batch,
                                         shuffle=False)

test_loader = torch.utils.data.DataLoader(PopulationDataset(division=args.division,
                                                            T=args.T,
                                                            type='test'),
                                          batch_size=args.test_batch,
                                          shuffle=False)

model = Model(division=args.division,
              C=args.C,
              K=args.K,
              T=args.T)

torch.cuda.set_device(0)
torch.manual_seed(args.seed)
model = model.cuda()

model.A = Variable(model.A).cuda()
model.F = Variable(model.F).cuda()
model.A_mean = Variable(model.A_mean).cuda()
model.I = Variable(model.I).cuda()

label = np.load('poi/' + args.division + '_label.npy').astype(np.int64)
label_idx = np.load('poi/' + args.division + '_idx.npy').astype(np.int64)
label_weight = np.load('poi/' + args.division + '_weight.npy').astype(np.float32)

label = torch.from_numpy(label[label_idx])
label_idx = torch.from_numpy(label_idx)
label_weight = torch.from_numpy(label_weight)
label = Variable(label).cuda()
label_idx = Variable(label_idx).cuda()
label_weight = Variable(label_weight).cuda()

if args.load > 0:
    model.load_state_dict(torch.load('checkpoint/epoch_' + str(args.load) + '.tar'))

optimizer = optim.Adam(model.parameters(), lr=args.lr)
criterion = nn.MSELoss(size_average=True)
loss = np.zeros((3, args.epoch), dtype=np.float32)
val_loss_min = 100
test_loss_min = 100

for epoch in range(1 + args.load, args.epoch + 1 + args.load):
    start = time()
    train_loss = train(model, train_loader, optimizer, criterion, epoch, label, label_idx, label_weight, args.theta)
    val_loss = val(model, val_loader, criterion, epoch, label_idx)
    test_loss = test(model, test_loader, criterion, epoch, label_idx)
    stop = time()
    print ("Time used: %.0f" % (stop - start))
    loss[0, epoch - args.load - 1] = train_loss
    loss[1, epoch - args.load - 1] = val_loss
    loss[2, epoch - args.load - 1] = test_loss
    if val_loss < val_loss_min:
        val_loss_min = val_loss
        test_loss_min = test_loss
    if epoch % 100 == 0:
        torch.save(model.state_dict(), 'checkpoint/epoch_' + str(epoch) + '.tar')

np.save('checkpoint/loss.npy', loss)
print("Val RMSE: %.4f \tTest RMSE: %.4f\n" % (
    val_loss_min, test_loss_min))

/home/rodr/env38/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))
/home/rodr/code/mobility-baselines/3D-DGCN-master/gcn.py:57: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  Z = self.Softmax(Y2)


Epoch: 1 [0/1762 (0%)]	Loss: 17.612080	Label Loss: 2.210205
Epoch: 1 [640/1762 (36%)]	Loss: 13.418503	Label Loss: 2.175114
Epoch: 1 [1280/1762 (71%)]	Loss: 18.560494	Label Loss: 2.162655

Epoch: 1 	Val Loss: 19.890282
Epoch: 1 	Test Loss: 17.774647

Time used: 1
Epoch: 2 [0/1762 (0%)]	Loss: 14.775474	Label Loss: 2.117697
Epoch: 2 [640/1762 (36%)]	Loss: 17.723562	Label Loss: 2.097046
Epoch: 2 [1280/1762 (71%)]	Loss: 15.755142	Label Loss: 2.072866

Epoch: 2 	Val Loss: 18.366209
Epoch: 2 	Test Loss: 16.193601

Time used: 1
Epoch: 3 [0/1762 (0%)]	Loss: 15.707941	Label Loss: 2.061525
Epoch: 3 [640/1762 (36%)]	Loss: 17.777892	Label Loss: 2.045019
Epoch: 3 [1280/1762 (71%)]	Loss: 15.027293	Label Loss: 2.022030

Epoch: 3 	Val Loss: 17.253261
Epoch: 3 	Test Loss: 15.202509

Time used: 1
Epoch: 4 [0/1762 (0%)]	Loss: 13.505002	Label Loss: 2.000395
Epoch: 4 [640/1762 (36%)]	Loss: 13.290968	Label Loss: 1.994641
Epoch: 4 [1280/1762 (71%)]	Loss: 13.747518	Label Loss: 1.980594

Epoch: 4 	Val Loss: 16.

Epoch: 32 [640/1762 (36%)]	Loss: 11.744823	Label Loss: 1.588379
Epoch: 32 [1280/1762 (71%)]	Loss: 12.614817	Label Loss: 1.533792

Epoch: 32 	Val Loss: 12.002448
Epoch: 32 	Test Loss: 9.728051

Time used: 1
Epoch: 33 [0/1762 (0%)]	Loss: 10.296696	Label Loss: 1.531360
Epoch: 33 [640/1762 (36%)]	Loss: 11.799423	Label Loss: 1.537148
Epoch: 33 [1280/1762 (71%)]	Loss: 11.292571	Label Loss: 1.513937

Epoch: 33 	Val Loss: 12.101423
Epoch: 33 	Test Loss: 9.779365

Time used: 1
Epoch: 34 [0/1762 (0%)]	Loss: 11.584651	Label Loss: 1.509512
Epoch: 34 [640/1762 (36%)]	Loss: 9.651443	Label Loss: 1.487226
Epoch: 34 [1280/1762 (71%)]	Loss: 8.994326	Label Loss: 1.501613

Epoch: 34 	Val Loss: 12.179383
Epoch: 34 	Test Loss: 9.923064

Time used: 1
Epoch: 35 [0/1762 (0%)]	Loss: 10.089330	Label Loss: 1.477589
Epoch: 35 [640/1762 (36%)]	Loss: 9.526337	Label Loss: 1.489280
Epoch: 35 [1280/1762 (71%)]	Loss: 10.357015	Label Loss: 1.481767

Epoch: 35 	Val Loss: 12.042618
Epoch: 35 	Test Loss: 9.604953

Time used

Epoch: 63 [640/1762 (36%)]	Loss: 11.192964	Label Loss: 1.163718
Epoch: 63 [1280/1762 (71%)]	Loss: 10.940051	Label Loss: 1.179958

Epoch: 63 	Val Loss: 11.400210
Epoch: 63 	Test Loss: 9.149861

Time used: 1
Epoch: 64 [0/1762 (0%)]	Loss: 8.901832	Label Loss: 1.158366
Epoch: 64 [640/1762 (36%)]	Loss: 10.753482	Label Loss: 1.177938
Epoch: 64 [1280/1762 (71%)]	Loss: 9.939625	Label Loss: 1.160264

Epoch: 64 	Val Loss: 11.322260
Epoch: 64 	Test Loss: 9.464971

Time used: 1
Epoch: 65 [0/1762 (0%)]	Loss: 8.057605	Label Loss: 1.164510
Epoch: 65 [640/1762 (36%)]	Loss: 9.596659	Label Loss: 1.156564
Epoch: 65 [1280/1762 (71%)]	Loss: 8.580628	Label Loss: 1.158992

Epoch: 65 	Val Loss: 11.337999
Epoch: 65 	Test Loss: 9.089378

Time used: 1
Epoch: 66 [0/1762 (0%)]	Loss: 9.792744	Label Loss: 1.164504
Epoch: 66 [640/1762 (36%)]	Loss: 9.255188	Label Loss: 1.141393
Epoch: 66 [1280/1762 (71%)]	Loss: 8.995903	Label Loss: 1.151219

Epoch: 66 	Val Loss: 11.014565
Epoch: 66 	Test Loss: 8.922794

Time used: 1
E

Epoch: 94 [640/1762 (36%)]	Loss: 8.945726	Label Loss: 0.994764
Epoch: 94 [1280/1762 (71%)]	Loss: 9.145617	Label Loss: 1.004702

Epoch: 94 	Val Loss: 11.092093
Epoch: 94 	Test Loss: 8.940401

Time used: 1
Epoch: 95 [0/1762 (0%)]	Loss: 8.855284	Label Loss: 0.973031
Epoch: 95 [640/1762 (36%)]	Loss: 11.795485	Label Loss: 0.955279
Epoch: 95 [1280/1762 (71%)]	Loss: 10.987978	Label Loss: 1.005443

Epoch: 95 	Val Loss: 11.189836
Epoch: 95 	Test Loss: 8.913062

Time used: 1
Epoch: 96 [0/1762 (0%)]	Loss: 9.516347	Label Loss: 0.983635
Epoch: 96 [640/1762 (36%)]	Loss: 9.384496	Label Loss: 0.982420
Epoch: 96 [1280/1762 (71%)]	Loss: 9.556126	Label Loss: 0.988429

Epoch: 96 	Val Loss: 11.389013
Epoch: 96 	Test Loss: 9.238483

Time used: 1
Epoch: 97 [0/1762 (0%)]	Loss: 9.404527	Label Loss: 0.982000
Epoch: 97 [640/1762 (36%)]	Loss: 13.122085	Label Loss: 0.979307
Epoch: 97 [1280/1762 (71%)]	Loss: 9.945328	Label Loss: 1.005571

Epoch: 97 	Val Loss: 11.208011
Epoch: 97 	Test Loss: 8.870175

Time used: 1
E

...

Epoch: 1472 [640/1762 (36%)]	Loss: 8.396654	Label Loss: 0.191931
Epoch: 1472 [1280/1762 (71%)]	Loss: 8.191964	Label Loss: 0.184651

Epoch: 1472 	Val Loss: 9.826482
Epoch: 1472 	Test Loss: 7.740045

Time used: 1
Epoch: 1473 [0/1762 (0%)]	Loss: 7.271606	Label Loss: 0.190371
Epoch: 1473 [640/1762 (36%)]	Loss: 7.136303	Label Loss: 0.185449
Epoch: 1473 [1280/1762 (71%)]	Loss: 8.566349	Label Loss: 0.174233

Epoch: 1473 	Val Loss: 9.894267
Epoch: 1473 	Test Loss: 7.938389

Time used: 1
Epoch: 1474 [0/1762 (0%)]	Loss: 7.737792	Label Loss: 0.185356
Epoch: 1474 [640/1762 (36%)]	Loss: 8.202516	Label Loss: 0.188683
Epoch: 1474 [1280/1762 (71%)]	Loss: 8.032853	Label Loss: 0.182134

Epoch: 1474 	Val Loss: 10.073419
Epoch: 1474 	Test Loss: 7.939101

Time used: 1
Epoch: 1475 [0/1762 (0%)]	Loss: 7.752645	Label Loss: 0.184053
Epoch: 1475 [640/1762 (36%)]	Loss: 8.148753	Label Loss: 0.180943
Epoch: 1475 [1280/1762 (71%)]	Loss: 8.012055	Label Loss: 0.186192

Epoch: 1475 	Val Loss: 9.970856
Epoch: 1475 	Tes

Epoch: 1502 [1280/1762 (71%)]	Loss: 6.346565	Label Loss: 0.190404

Epoch: 1502 	Val Loss: 9.788538
Epoch: 1502 	Test Loss: 7.789690

Time used: 1
Epoch: 1503 [0/1762 (0%)]	Loss: 7.029803	Label Loss: 0.174776
Epoch: 1503 [640/1762 (36%)]	Loss: 8.536328	Label Loss: 0.180161
Epoch: 1503 [1280/1762 (71%)]	Loss: 7.896578	Label Loss: 0.182395

Epoch: 1503 	Val Loss: 9.907546
Epoch: 1503 	Test Loss: 7.913074

Time used: 1
Epoch: 1504 [0/1762 (0%)]	Loss: 8.119555	Label Loss: 0.176040
Epoch: 1504 [640/1762 (36%)]	Loss: 7.338000	Label Loss: 0.172917
Epoch: 1504 [1280/1762 (71%)]	Loss: 8.290980	Label Loss: 0.175745

Epoch: 1504 	Val Loss: 9.745685
Epoch: 1504 	Test Loss: 7.814477

Time used: 1
Epoch: 1505 [0/1762 (0%)]	Loss: 8.205064	Label Loss: 0.180120
Epoch: 1505 [640/1762 (36%)]	Loss: 7.594184	Label Loss: 0.186859
Epoch: 1505 [1280/1762 (71%)]	Loss: 7.178561	Label Loss: 0.181174

Epoch: 1505 	Val Loss: 9.771611
Epoch: 1505 	Test Loss: 7.703242

Time used: 1
Epoch: 1506 [0/1762 (0%)]	Loss: 7.6


Epoch: 1532 	Val Loss: 9.874185
Epoch: 1532 	Test Loss: 7.769868

Time used: 1
Epoch: 1533 [0/1762 (0%)]	Loss: 7.981834	Label Loss: 0.180429
Epoch: 1533 [640/1762 (36%)]	Loss: 8.239962	Label Loss: 0.188801
Epoch: 1533 [1280/1762 (71%)]	Loss: 7.236056	Label Loss: 0.182805

Epoch: 1533 	Val Loss: 9.840291
Epoch: 1533 	Test Loss: 7.762823

Time used: 1
Epoch: 1534 [0/1762 (0%)]	Loss: 7.245155	Label Loss: 0.181368
Epoch: 1534 [640/1762 (36%)]	Loss: 7.997505	Label Loss: 0.174418
Epoch: 1534 [1280/1762 (71%)]	Loss: 8.476454	Label Loss: 0.174725

Epoch: 1534 	Val Loss: 9.949696
Epoch: 1534 	Test Loss: 7.911166

Time used: 1
Epoch: 1535 [0/1762 (0%)]	Loss: 7.207429	Label Loss: 0.174431
Epoch: 1535 [640/1762 (36%)]	Loss: 7.345633	Label Loss: 0.173470
Epoch: 1535 [1280/1762 (71%)]	Loss: 7.310702	Label Loss: 0.175488

Epoch: 1535 	Val Loss: 9.803815
Epoch: 1535 	Test Loss: 7.745462

Time used: 1
Epoch: 1536 [0/1762 (0%)]	Loss: 8.016113	Label Loss: 0.173329
Epoch: 1536 [640/1762 (36%)]	Loss: 8.25

Epoch: 1563 [640/1762 (36%)]	Loss: 7.963316	Label Loss: 0.172803
Epoch: 1563 [1280/1762 (71%)]	Loss: 6.754775	Label Loss: 0.174503

Epoch: 1563 	Val Loss: 9.711707
Epoch: 1563 	Test Loss: 7.769272

Time used: 1
Epoch: 1564 [0/1762 (0%)]	Loss: 7.581281	Label Loss: 0.177094
Epoch: 1564 [640/1762 (36%)]	Loss: 7.731339	Label Loss: 0.177054
Epoch: 1564 [1280/1762 (71%)]	Loss: 7.665174	Label Loss: 0.173769

Epoch: 1564 	Val Loss: 9.795930
Epoch: 1564 	Test Loss: 7.755904

Time used: 1
Epoch: 1565 [0/1762 (0%)]	Loss: 7.053040	Label Loss: 0.172467
Epoch: 1565 [640/1762 (36%)]	Loss: 7.595065	Label Loss: 0.169913
Epoch: 1565 [1280/1762 (71%)]	Loss: 5.832244	Label Loss: 0.176270

Epoch: 1565 	Val Loss: 9.725755
Epoch: 1565 	Test Loss: 7.775578

Time used: 1
Epoch: 1566 [0/1762 (0%)]	Loss: 7.718530	Label Loss: 0.167996
Epoch: 1566 [640/1762 (36%)]	Loss: 6.896141	Label Loss: 0.174030
Epoch: 1566 [1280/1762 (71%)]	Loss: 8.329859	Label Loss: 0.167423

Epoch: 1566 	Val Loss: 9.685298
Epoch: 1566 	Test

Epoch: 1593 [1280/1762 (71%)]	Loss: 7.486375	Label Loss: 0.180513

Epoch: 1593 	Val Loss: 9.767079
Epoch: 1593 	Test Loss: 7.733242

Time used: 1
Epoch: 1594 [0/1762 (0%)]	Loss: 8.121359	Label Loss: 0.171672
Epoch: 1594 [640/1762 (36%)]	Loss: 6.949965	Label Loss: 0.175253
Epoch: 1594 [1280/1762 (71%)]	Loss: 8.449793	Label Loss: 0.175137

Epoch: 1594 	Val Loss: 9.776522
Epoch: 1594 	Test Loss: 7.790649

Time used: 1
Epoch: 1595 [0/1762 (0%)]	Loss: 7.735147	Label Loss: 0.158931
Epoch: 1595 [640/1762 (36%)]	Loss: 9.506391	Label Loss: 0.173997
Epoch: 1595 [1280/1762 (71%)]	Loss: 6.549069	Label Loss: 0.163375

Epoch: 1595 	Val Loss: 9.834189
Epoch: 1595 	Test Loss: 7.805012

Time used: 1
Epoch: 1596 [0/1762 (0%)]	Loss: 8.030363	Label Loss: 0.168487
Epoch: 1596 [640/1762 (36%)]	Loss: 6.947620	Label Loss: 0.182014
Epoch: 1596 [1280/1762 (71%)]	Loss: 7.808929	Label Loss: 0.174131

Epoch: 1596 	Val Loss: 9.709712
Epoch: 1596 	Test Loss: 7.798204

Time used: 1
Epoch: 1597 [0/1762 (0%)]	Loss: 8.0

In [4]:
model.eval()
test_loss = 0
for batch_idx, (data, x_t, target) in enumerate(test_loader):
    data, x_t, target = Variable(data).cuda(), Variable(x_t).cuda(), Variable(target).cuda()
    output, _ = model(data, x_t, label_idx)
    loss = criterion(output, target)
    test_loss += loss.item() * len(data)
print ("Epoch: %d \tTest Loss: %.6f\n" % (
    epoch, sqrt(test_loss / len(test_loader.dataset))))

Epoch: 1600 	Test Loss: 7.684051



In [5]:
output.shape

torch.Size([28, 2, 82])

In [6]:
for loc in range(output.shape[2]):
    print(loc, output[:,0,loc].std().item())

0 34.09896469116211
1 43.4434928894043
2 16.223949432373047
3 39.42856216430664
4 13.306113243103027
5 11.089686393737793
6 9.036905288696289
7 12.078822135925293
8 21.155597686767578
9 46.839351654052734
10 53.569026947021484
11 33.55111312866211
12 10.13471794128418
13 15.231658935546875
14 12.880043029785156
15 15.048741340637207
16 21.177288055419922
17 3.1218421459198
18 35.57676696777344
19 7.629894256591797
20 12.096185684204102
21 10.087974548339844
22 16.13181495666504
23 9.553046226501465
24 7.206345558166504
25 8.700483322143555
26 5.441711902618408
27 37.48239517211914
28 22.36681365966797
29 5.682291507720947
30 15.176478385925293
31 4.575870990753174
32 15.066068649291992
33 66.42543029785156
34 19.853899002075195
35 16.51815414428711
36 11.795730590820312
37 17.627979278564453
38 36.74932861328125
39 27.703386306762695
40 15.733436584472656
41 26.644357681274414
42 29.67157745361328
43 17.557693481445312
44 18.678607940673828
45 23.296464920043945
46 33.664363861083984
4